In [141]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt     # Data Visualisation
import torch
import time
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [142]:
#import data from .csv file of guithub repository
housing = pd.read_csv('Housing.csv')
housing

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [143]:
# List of variables to map (string to numerical values)
varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, "no": 0})

# Applying the function to the housing list
housing[varlist] = housing[varlist].apply(binary_map)
           
# Check the housing dataframe now
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [144]:
#Choosing the values of the columns as per the question
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
new_df = housing[num_vars]
new_df

,price,area,bedrooms,bathrooms,stories,parking
0,13300000,7420,4,2,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,3,2,2,2
3,12215000,7500,4,2,2,3
4,11410000,7420,4,1,2,2
...,...,...,...,...,...,...
540,1820000,3000,2,1,1,2
541,1767150,2400,3,1,1,0
542,1750000,3620,2,1,1,0
543,1750000,2910,3,1,1,0


In [145]:
new_df.shape

(545, 6)

In [146]:
#Scaling the data by Normalization

scaler = MinMaxScaler()        # input Normalization
new_df[num_vars] = scaler.fit_transform(new_df[num_vars])
new_df.head(10)

,price,area,bedrooms,bathrooms,stories,parking
0,1.000000,0.396564,0.6,0.333333,0.666667,0.666667
1,0.909091,0.502405,0.6,1.000000,1.000000,1.000000
2,0.909091,0.571134,0.4,0.333333,0.333333,0.666667
3,0.906061,0.402062,0.6,0.333333,0.333333,1.000000
4,0.836364,0.396564,0.6,0.000000,0.333333,0.666667
5,0.787879,0.402062,0.4,0.666667,0.000000,0.666667
6,0.727273,0.476289,0.6,0.666667,1.000000,0.666667
7,0.727273,1.000000,0.8,0.666667,0.333333,0.000000
8,0.703030,0.443299,0.6,0.000000,0.333333,0.666667
9,0.696970,0.281787,0.4,0.333333,1.000000,0.333333


In [147]:
#Splitting the dataset into labels and Ground truth
X = new_df.iloc[:, 1:6].values
Y = new_df.iloc[:, 0].values

In [148]:
#Converting into tensors
X = torch.tensor(X)
Y = torch.tensor(Y)

In [149]:
#80% (training) and 20% (validation) split
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

#For efficient training shuffling the indices of the original dataset
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [150]:
train_X = X[train_indices]
train_Y = Y[train_indices]

#Adding an extra dimension, to make the labels from (len,) to (len,1)
train_Y = train_Y.unsqueeze(1)

val_X = X[val_indices]
val_Y = Y[val_indices]

#Adding an extra dimension, to make the labels from (len,) to (len,1)
val_Y = val_Y.unsqueeze(1)

print(train_X.shape)
print(train_Y.shape)
print(val_X.shape)
print(val_Y.shape)

torch.Size([436, 5])
torch.Size([436, 1])
torch.Size([109, 5])
torch.Size([109, 1])


In [151]:
import torch
import torch.nn as nn
import torch.optim as optim

# Fully connected neural network having one hidden layer with 8 nodes

model = nn.Sequential(
            nn.Linear(5,8),
            nn.ReLU(), 
            nn.LogSoftmax(dim=1))

model = model.float()
learning_rate = 0.01

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

out = model(train_X.float())


loss_fn = nn.MSELoss()

n_epochs = 200
correct = 0
total = 0

#Defining the training loop 

for epoch in range(n_epochs+1):
        training_start_time = time.time()
        out = model(train_X.float())
        loss = loss_fn(out,train_Y.float())
        
        out_v= model(val_X.float()) # <1>
        val_loss = loss_fn(out_v, val_Y.float())
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch==0 or epoch % 20 ==0:
            print("Epoch: %d, Training Loss: %f, Validation Loss: %f" % (epoch, float(loss), float(val_loss)))
        _, predicted = torch.max(out, dim=1)
        total += val_Y.shape[0]
        correct += int((predicted == train_Y).sum())
        
print("Accuracy: %f" % (correct / total))

# Pringting out the training time
print('Training finished, took {:.2f}s'.format(time.time() - training_start_time))  

Epoch: 0, Training Loss: 5.576751, Validation Loss: 5.442920
Epoch: 20, Training Loss: 5.567597, Validation Loss: 5.434439
Epoch: 40, Training Loss: 5.560877, Validation Loss: 5.428255
Epoch: 60, Training Loss: 5.555753, Validation Loss: 5.423554
Epoch: 80, Training Loss: 5.551794, Validation Loss: 5.419948
Epoch: 100, Training Loss: 5.548697, Validation Loss: 5.417112
Epoch: 120, Training Loss: 5.546229, Validation Loss: 5.414848
Epoch: 140, Training Loss: 5.544240, Validation Loss: 5.413029
Epoch: 160, Training Loss: 5.542561, Validation Loss: 5.411497
Epoch: 180, Training Loss: 5.541170, Validation Loss: 5.410233
Epoch: 200, Training Loss: 5.540003, Validation Loss: 5.409183
Accuracy: 3.184582
Training finished, took 0.00s


In [152]:
import torch
import torch.nn as nn
import torch.optim as optim

# Fully connected neural network having one hidden layer with 8 nodes

model = nn.Sequential(
            nn.Linear(5,2),
            nn.ReLU(), 
            nn.LogSoftmax(dim=1))

model = model.float()
learning_rate = 0.01

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

out = model(train_X.float())


loss_fn = nn.MSELoss()

n_epochs = 200
correct = 0
total = 0

#Defining the training loop 

for epoch in range(n_epochs+1):
        training_start_time = time.time()
        out = model(train_X.float())
        loss = loss_fn(out,train_Y.float())
        
        out_v= model(val_X.float()) # <1>
        val_loss = loss_fn(out_v, val_Y.float())
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch==0 or epoch % 20 ==0:
            print("Epoch: %d, Training Loss: %f, Validation Loss: %f" % (epoch, float(loss), float(val_loss)))
        _, predicted = torch.max(out, dim=1)
        total += val_Y.shape[0]
        correct += int((predicted == train_Y).sum())
        
print("Accuracy: %f" % (correct / total))

# Pringting out the training time
print('Training finished, took {:.2f}s'.format(time.time() - training_start_time))   

Epoch: 0, Training Loss: 1.023157, Validation Loss: 0.957786
Epoch: 20, Training Loss: 0.993257, Validation Loss: 0.930469
Epoch: 40, Training Loss: 0.976037, Validation Loss: 0.914945
Epoch: 60, Training Loss: 0.966056, Validation Loss: 0.906072
Epoch: 80, Training Loss: 0.960142, Validation Loss: 0.900841
Epoch: 100, Training Loss: 0.956535, Validation Loss: 0.897673
Epoch: 120, Training Loss: 0.954267, Validation Loss: 0.895697
Epoch: 140, Training Loss: 0.952779, Validation Loss: 0.894419
Epoch: 160, Training Loss: 0.951763, Validation Loss: 0.893559
Epoch: 180, Training Loss: 0.951053, Validation Loss: 0.892967
Epoch: 200, Training Loss: 0.950545, Validation Loss: 0.892550
Accuracy: 11.610982
Training finished, took 0.00s


In [153]:
import torch
import torch.nn as nn
import torch.optim as optim

# # Fully connected neural network with three hidden layers

model2 = nn.Sequential(
             nn.Linear(5,16),
            nn.ReLU(),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.Linear(8,2),
            nn.ReLU(),    
            nn.LogSoftmax(dim=1))
model = model.float()
learning_rate = 0.01

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

out = model2(train_X.float())


loss_fn = nn.MSELoss()

n_epochs = 200
correct = 0
total = 0

#Defining the training loop 

for epoch in range(n_epochs+1):
        training_start_time = time.time()
        out = model2(train_X.float())
        loss = loss_fn(out,train_Y.float())
        
        out_v= model2(val_X.float()) # <1>
        val_loss = loss_fn(out_v, val_Y.float())
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch==0 or epoch % 20 ==0:
            print("Epoch: %d, Training Loss: %f, Validation Loss: %f" % (epoch, float(loss), float(val_loss)))
        _, predicted = torch.max(out, dim=1)
        total += val_Y.shape[0]
        correct += int((predicted == train_Y).sum())
        
print("Accuracy: %f" % (correct / total))

# Pringting out the training time
print('Training finished, took {:.2f}s'.format(time.time() - training_start_time))   

Epoch: 0, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 20, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 40, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 60, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 80, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 100, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 120, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 140, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 160, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 180, Training Loss: 0.974788, Validation Loss: 0.917030
Epoch: 200, Training Loss: 0.974788, Validation Loss: 0.917030
Accuracy: 12.000000
Training finished, took 0.00s
